## Goal: Get Min/Max prices for all SKUs, based on spreadsheet.

#### Inputs: 
    * Inventory Health report from Amazon (contains age and stock level per SKU)
    * FBA inventory report
    * CDGD sourcing spreadsheet (contains seasonality classification and min/max prices)
    
#### Outputs: 
    * list of SKUs, min and max prices for each

In [1]:
import pandas as pd
import numpy as np
import pyisbn #module for converting ISBN-13 to ISBN-10 and vice versa


invhealth = pd.read_csv('inventory_health_aug19.csv',engine='python')
FBAinv = pd.read_csv('FBA_inventory_aug19.csv',engine='python')
CDGD_inv = pd.read_csv('CDGD_inventory.csv')

In [145]:
invhealth

,snapshot-date,sku,fnsku,asin,product-name,condition,sales-rank,product-group,total-quantity,sellable-quantity,...,qty-in-long-term-storage-program,qty-with-removals-in-progress,projected-ltsf-12-mo,per-unit-volume,is-hazmat,in-bound-quantity,asin-limit,inbound-recommend-quantity,qty-to-be-charged-ltsf-6-mo,projected-ltsf-6-mo
0,2020-08-17T00:00:00-07:00,CP_July25-004,X002LI1OLJ,1285741552,Calculus: Early Transcendentals,Used,1313,gl_book,2,2,...,0,NaN,0.00,0.114460,N,0,0,0,0,0.0
1,2020-08-17T00:00:00-07:00,CP_July25-006,X002LHXEJ5,0134243064,"Biological Science, Books a la Carte Edition (...",Used,34309,gl_book,1,1,...,0,NaN,0.00,0.146412,N,0,0,0,0,0.0
2,2020-08-17T00:00:00-07:00,CP_July25-003,X002LHSA03,0393680363,Essential Cell Biology (Fifth Edition),Used,9890,gl_book,3,3,...,0,NaN,0.00,0.065130,N,0,0,0,0,0.0
3,2020-08-17T00:00:00-07:00,CP_July25-002,X002LHS9H7,1305077695,"Those Who Can, Teach",Used,5766,gl_book,3,3,...,0,NaN,0.00,0.042575,N,0,0,0,0,0.0
4,2020-08-17T00:00:00-07:00,nar_july14-001,X002L2F1MN,146413538X,Quantitative Chemical Analysis,Used,5124,gl_book,3,3,...,0,NaN,0.00,0.095622,N,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2020-08-17T00:00:00-07:00,NZ-TE0Y-DRRC,X001WYMHK5,0195331451,Modern Digital and Analog Communication System...,Used,603329,gl_book,2,2,...,2,NaN,1.04,0.075000,N,0,0,0,0,0.0
224,2020-08-17T00:00:00-07:00,jul14-12,X001U7SX07,0195331451,Modern Digital and Analog Communication System...,Used,603329,gl_book,1,1,...,1,NaN,0.52,0.075000,N,0,0,0,0,0.0
225,2020-08-17T00:00:00-07:00,jul14-5,X001U7QS95,0132553287,Management (3rd Edition),No Listing,325198,gl_book,1,1,...,0,NaN,0.38,0.054351,N,0,0,0,0,0.0
226,2020-08-17T00:00:00-07:00,0L6-S2A-6YR,X001RKZJE5,0757552013,"Russian Stage One: Live from Russia, Vol. 1 (B...",Used,211684,gl_book,8,8,...,0,NaN,0.00,0.102125,N,0,0,0,0,0.0


In [51]:
FBAinv

,sku,fnsku,asin,product-name,condition,your-price,mfn-listing-exists,mfn-fulfillable-quantity,afn-listing-exists,afn-warehouse-quantity,...,afn-unsellable-quantity,afn-reserved-quantity,afn-total-quantity,per-unit-volume,afn-inbound-working-quantity,afn-inbound-shipped-quantity,afn-inbound-receiving-quantity,afn-researching-quantity,afn-reserved-future-supply,afn-future-supply-buyable
0,0G-ADX1-OU1X,X0024K9LTV,0131429175,Discrete-Event Simulation: A First Course,UsedGood,86.42,No,NaN,Yes,0,...,0,0,0,0.04,0,0,0,0,0,0
1,0L6-S2A-6YR,X001RKZJE5,0757552013,"Russian Stage One: Live from Russia, Vol. 1 (B...",UsedAcceptable,73.67,No,NaN,Yes,8,...,0,1,8,0.10,0,0,0,0,0,0
2,0U-55S2-9AHF,X0024SWFTB,0321981227,Human Physiology: An Integrated Approach (7th ...,UsedAcceptable,54.99,No,NaN,Yes,0,...,0,0,0,0.09,0,0,0,0,0,0
3,34A-MXC-2AN,X001NCQ3QZ,1305580346,General Chemistry - Standalone book,UsedGood,74.99,No,NaN,Yes,0,...,0,0,0,0.10,0,0,0,0,0,0
4,36G-XE0-HC4,X001C02REZ,0495826723,Student Solutions Manual for Blanchard/Devaney...,UsedGood,19.99,No,NaN,Yes,0,...,0,0,0,0.03,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,r_apr14-039,X00248E9YF,0073403717,Anatomy & Physiology: The Unity of Form and Fu...,UsedAcceptable,15.62,No,NaN,Yes,0,...,0,0,0,0.12,0,0,0,0,0,0
533,r_apr29-004,X002500BB7,0073403717,Anatomy & Physiology: The Unity of Form and Fu...,UsedAcceptable,15.62,No,NaN,Yes,0,...,0,0,0,0.12,0,0,0,0,0,0
534,r_may15-021,X0026CCCA7,1285079701,"Aplia, 1 term Printed Access Card for Gravette...",UsedAcceptable,29.99,No,NaN,Yes,1,...,0,0,1,0.00,0,0,0,0,0,0
535,r_may2-021,X00253C725,0073403717,Anatomy & Physiology: The Unity of Form and Fu...,UsedAcceptable,15.62,No,NaN,Yes,0,...,0,0,0,0.12,0,0,0,0,0,0


In [52]:
CDGD_inv

,Title,updated 2020?,Cover Art,ISBN-13,DealOz Link,Amazon Link,New ed exists?,Seasonality,Season,Seasonality notes,Trend notes,Popularity notes,Approved buy price,Approved list price,List price MUST not go below:,Notes,Keepa
0,Student's Study Guide and Solutions Manual for...,NaN,rainbow swirl loose leaf,978134066585,https://dealoz.com/prod?cat=&catby=&class=new~...,https://smile.amazon.com/Students-Solutions-Ma...,N,H,S+/W,NaN,NaN,NaN,60.0,115.0,NaN,NaN,NaN
1,iGenetics: A Molecular Approach (3rd Edition),y,NaN,9780321569769,https://dealoz.com/prod?cat=&catby=&class=new~...,https://smile.amazon.com/iGenetics-Molecular-A...,N,H,NaN,NaN,NaN,NaN,20.0,60.0,NaN,NaN,NaN
2,Invitation to the Life Span,y,little girl & grandpa,9781319015886,https://dealoz.com/New-Used-Books/Invitation-T...,https://www.amazon.com/Invitation-Life-Kathlee...,Y,H,none,NaN,Downward trend,NaN,20.0,70.0,40,NaN,NaN
3,Theories of Counseling and Psychotherapy: Syst...,y,NaN,9780132851701,https://dealoz.com/prod?cat=&catby=&class=new~...,https://www.amazon.com/Theories-Counseling-Psy...,N,H,none,NaN,NaN,NaN,40.0,80.0,67,NaN,NaN
4,Materials Science and Engineering: An Introduc...,y,molecular representation,9781118324578,https://dealoz.com/prod?cat=&catby=&class=new~...,https://www.amazon.com/Materials-Science-Engin...,N,H,S,Favorite,NaN,NaN,45.0,120.0,73,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,Farm Management 8th Edition by Ronald Kay,NaN,NaN,9780073400945,https://dealoz.com/prod?cat=&catby=&class=new~...,https://www.amazon.com/Farm-Management-Ronald-...,Y,M,NaN,NaN,NaN,NaN,NaN,NaN,#REF!,NaN,NaN
666,Emergency Medical Responder: Your First Respon...,NaN,NaN,9781449693008,https://dealoz.com/prod?cat=&catby=&class=new~...,https://www.amazon.com/dp/1449693008,Y,M,NaN,NaN,NaN,NaN,NaN,NaN,#REF!,NaN,NaN
667,Drug Use and Abuse: A Comprehensive Introducti...,NaN,NaN,9781285070278,https://dealoz.com/prod?cat=&catby=&class=new~...,https://www.amazon.com/dp/1285070275,Y,M,NaN,NaN,NaN,NaN,NaN,NaN,#REF!,NaN,NaN
668,"Options, Futures, and Other Derivatives (10th ...",y,globe,9780134472089,https://dealoz.com/prod?cat=&catby=&class=new~...,https://smile.amazon.com/Options-Futures-Other...,N,NaN,NaN,NaN,NaN,NaN,75.0,125.0,NaN,NaN,NaN


In [1]:
# From list of SKUs, get their min/max price from spreadsheet
SKUs = FBAinv['sku'].to_list()
ASINs = FBAinv['asin'].to_list()
#ISBNs = [pyisbn.convert(asin) for asin in ASINs]

ISBNs = list()

for asin in ASINs:
    try: 
        ISBNs.append(pyisbn.convert(asin))
    except:
        ISBNs.append('UNKNOWN')


min_prices = 4.99*np.ones(len(ISBNs))
max_prices = 299.99*np.ones(len(ISBNs))

df = pd.DataFrame({
    'sku': SKUs,
    'ASIN': ASINs,
    'ISBN': ISBNs,
    'min_price': min_prices,
    'max_price': max_prices
})

for isbn in ISBNs:
    min_price = CDGD_inv.loc[CDGD_inv['ISBN-13'] == isbn]['Approved list price'].to_list()
    if len(min_price)>0:
        df.at[df.index[df['ISBN']==isbn], 'min_price'] = min_price[0]


NameError: name 'FBAinv' is not defined

In [203]:
df.to_csv(r'C:\Users\Lyra\Documents\GitHub\CDGD\df_aug19.csv', index = False)